In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint
from functools import partial

from genpen.grower import *
import fn
from genpen.axicam import AxiCam

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=40
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
xpix = 3300
ypix = 4200

drawbox = box(0, 0, xpix, ypix)

all_circles = []
circle_to_fill = drawbox
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.5, 0.1)
# rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 30)
rads = [540, 270, 200, 100]
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 100
iter_max = 10
rad_mults = (0.5, 0.1)

n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 30)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False) and gp.get_rad(c) > min_rad]
        areas = np.array([c.area for c in not_yet_filled]) ** 2
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except KeyboardInterrupt:
        break
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
bottom_circles = MultiPolygon([c for c in all_circles if not getattr(c, 'filled', False)])
bottom_circles

In [ ]:
from rasterio import features

In [ ]:
db = gp.Shape(drawbox)

In [ ]:
dpmm = 20

In [ ]:
out_shape = (np.array((db.width, db.height)) * dpmm).astype(int)

In [ ]:
out_shape = (ypix, xpix)

In [ ]:
circles

In [ ]:
rastered = features.rasterize(list(all_circles), out_shape=out_shape)

In [ ]:
dfs = []
for ii, circle in enumerate(tqdm(all_circles)):
    row, col = np.where(features.rasterize([circle], out_shape=out_shape))
    _df = pd.DataFrame({'row': ii, 'col': jj})
    _df['poly'] = ii
    dfs.append(_df)

In [ ]:
df = pd.concat(dfs)

In [ ]:
df.to_csv('circle_inds.csv')

In [ ]:
all_circles

In [ ]:
dfs = []
for ii, circle in enumerate(tqdm(all_circles)):
    _df = pd.DataFrame({'x': circle.centroid.x, 'y': circle.centroid.y, 'rad': gp.get_rad(circle)}, index=[ii])
    dfs.append(_df)

df = pd.concat(dfs)

In [ ]:
df.to_csv('circle_summary.csv')

# flowbeam

In [ ]:
paper_size = '20x10 inches'
border:float=30
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:
from genpen.flow import field, particle

In [ ]:

poly = drawbox.buffer(-0)
pg = field.PerlinGrid(poly, lod=4, falloff=None, noise_scale=0.0063, noiseSeed=5)

In [ ]:
db = gp.Shape(poly)

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(pg.p, xstep=24, ystep=14)
start_points = []
for x,y in itertools.product(xcs,ycs):
    pos = Point(x+np.random.randn()*1.6, y+np.random.randn()*4.6)
    start_points.append(pos)

In [ ]:
# spiral_angle_max = np.pi * 10
# spiral_angle_min = 0
# spiral_angle_spacing = np.pi * 0.0501
# sp_angle_range = np.arange(spiral_angle_min, spiral_angle_max, spiral_angle_spacing)
# spiral_distances = np.linspace(0, 40, len(sp_angle_range))

# start_points = [Point(np.cos(a) * d, np.sin(a) * d) for a, d in zip(sp_angle_range, spiral_distances)]

In [ ]:
start_points = gp.make_like(MultiPoint(start_points), db.p)

In [ ]:
poly = db.p
pts = []
lss = []
n_steps = 55
for pt in tqdm(start_points):
    
    vp = particle.VectorParticle(pos=pt, grid=pg, stepsize=1, momentum_factor=np.random.uniform(0,0))
    for ii in range(n_steps):
        vp.step()
    if len(vp.pts) > 1:
        ls = gp.merge_LineStrings([LineString(vp.pts)])
    lss.append(ls)

In [ ]:
gp.merge_LineStrings(lss)

In [ ]:
buffer_gen = ss.uniform(loc=6, scale=5).rvs

In [ ]:
polys = []
all_polys = Polygon()
for ii, l in enumerate(tqdm(lss[:])):
    p = l.buffer(1, cap_style=2, join_style=2)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    visible_area = p.difference(all_polys).buffer(1e-6)
    polys.append(visible_area)
    all_polys = so.unary_union([all_polys, visible_area])

In [ ]:
gp.merge_Polygons(polys)